In [ ]:
import json

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.base import RunnableSerializable
from langchain_ollama.chat_models import ChatOllama

# 1. Setup

In [ ]:
llm = ChatOllama(model="llama3.1:8b", reasoning=False)

In [ ]:
@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y


@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x**y


@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x


tools = [add, subtract, multiply, exponentiate]
name2tool = {tool.name: tool.func for tool in tools}

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You're a helpful assistant. When answering a user's question "
                "you should first use one of the tools provided. After using a "
                "tool the tool output will be provided in the "
                "'scratchpad' below. If you have an answer in the "
                "scratchpad you should not use any more tools and "
                "instead answer directly to the user."
            ),
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 2. Custom Agent Executor

In [ ]:
class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", []),
            }
            | prompt
            | llm.bind_tools(tools)
        )

    def invoke(self, input: str) -> dict:
        count = 0
        agent_scratchpad = []

        while count < self.max_iterations:
            print(f"Iteration {count + 1} of {self.max_iterations}")
            agent_input = {
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad,
            }

            step = self.agent.invoke(agent_input)
            print(step)

            # Stop if no tool_calls – indicates final answer
            if not hasattr(step, "tool_calls") or not step.tool_calls:
                final_output = step.content if hasattr(step, "content") else str(step)
                break

            # Parse tool call
            tool_call = step.tool_calls[0]
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            tool_call_id = tool_call["id"]

            # Execute tool
            tool_out = name2tool[tool_name](**tool_args)

            # Append both tool call and response
            agent_scratchpad.append(step)
            agent_scratchpad.append(
                ToolMessage(content=f"{tool_out}", tool_call_id=tool_call_id)
            )

            print(f"{count}: {tool_name}({tool_args})")
            count += 1

        # Add final exchange to history
        self.chat_history.extend(
            [HumanMessage(content=input), AIMessage(content=final_output)]
        )

        return json.dumps({"answer": final_output})

In [ ]:
agent_executor = CustomAgentExecutor()

In [ ]:
agent_executor.invoke(input="What is 10 + 10")

# 3. Agent Executor

In [ ]:
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    max_iterations=3,
    verbose=True,
)

In [ ]:
agent_executor.invoke({"input": "What is 10 + 10", "chat_history": []})